# LLM + AI vs. LLM + AI (Qwen:32B)

In [10]:
import chess
from stockfish import Stockfish
from ollama import chat
from ollama import ChatResponse
import json
import random
import pandas as pd
import re

In [11]:
# STOCKFISH_PATH = r"C:\Users\nafis\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"

STOCKFISH_ELO = 3500
MOVE_LIMIT = 100
NUM_GAMES = 60

In [12]:
stockfish = Stockfish()

stockfish.update_engine_parameters({"Threads": 8, "Hash": 2048})

In [13]:
qwen_white_wins = 0
qwen_black_wins = 0
draws = 0

In [14]:
def valid_move_to_string(board):
    legal_moves = [
        board.san(move) for move in board.legal_moves]
    
    if not legal_moves:
        return "No legal moves."
    else:
        return ", ".join(legal_moves)

In [15]:
def get_move_history(board: chess.Board) -> str:
    """Return a formatted move history string like '1. e4 e5 2. Nf3 d5'."""
    temp_board = chess.Board()  # Start from the initial position.
    move_history = []
    
    # Iterate over the moves in the board's move stack.
    for i, move in enumerate(board.move_stack):
        # Get the SAN for the move in the current temporary board.
        move_san = temp_board.san(move)
        # Push the move so that the board state is updated.
        temp_board.push(move)
        
        # For white moves (even index), start a new entry with move number.
        if i % 2 == 0:
            move_history.append(f"{i//2 + 1}. {move_san}")
        # For black moves, append to the last white move entry.
        else:
            move_history[-1] += f" {move_san}"
    
    # Join all entries with a space.
    return " ".join(move_history)

In [16]:
def get_qwen_move(board:chess.Board, fen:str, temp=0.0):
    valid_moves = valid_move_to_string(board)

    move_history = get_move_history(board)
    
    stockfish.set_fen_position(board.fen())
    best_move = stockfish.get_best_move()
    best_move = board.san(chess.Move.from_uci(best_move))
    
    response: ChatResponse = chat(model='qwen:32b', messages=[
        {
            'role': 'system',
            'content': 'You are a grand master chess player. You have an AI assitant that plays chess. The AI will suggest you the best move for the current board state. You are instructed to be heavilty reliant on the AI suggestion.'
        },
        {
            'role': 'user',
            'content': f"""You are playing chess and it is your turn. This is the current state of the game. Use this to work out where the pieces are on the board:

FEN: {fen}

The possible set of legal moves are: 

Legal Moves: {valid_moves}

+(You have to choose one from the provided list. Do not choose a move that is not in the list.)

The best move suggested by your AI Assistant is {best_move}. You are heavily reliant on your AI Assistant.

The move history is: {move_history}.

Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.
{{
    "san": "The move in SAN format",
}}"""}], options={"temperature": temp})
    
    try:
        output = response.message.content
        # Use regex to extract the JSON object (brackets and everything inside)
        match = re.search(r'\{[\s\S]*?\}', output)
        if match:
            json_str = match.group(0)
            # Remove any trailing commas before the closing brace (e.g., after the colon with extra spaces)
            json_str = re.sub(r',\s*}', '}', json_str)
            parsed_json = json.loads(json_str)
            qwen_move = parsed_json["san"]
        else:
            raise ValueError("JSON object not found in response")
    except Exception as e:
        print(f"Wrong format given by Qwen. \nQwen Output: {response.message.content}")
        qwen_move = None

    print(f"Stockfish Suggested Move: {best_move}")

    return qwen_move

In [17]:
def get_random_move(board:chess.Board, fen:str, seed: int = 101):
    legal_moves = [
        board.san(move)
        for move in board.legal_moves
    ]

    random.seed(seed)

    # print(random.choice(legal_moves))
    return random.choice(legal_moves)

    # print(legal_moves[0])

In [18]:
for game_number in range(1, NUM_GAMES + 1):
    
    move_counter = 0

    print(f"Starting game {game_number}...")

    board = chess.Board()

    while not board.is_game_over() and move_counter <= MOVE_LIMIT:
        if board.turn == chess.WHITE and not board.is_game_over():
            qwen_move_white = get_qwen_move(board, board.fen(), 2.0)

            try:
                board.push_san(qwen_move_white)
            except Exception as e:
                print(f"Invalid move by Qwen white. Move: {qwen_move_white}")
                qwen_move_white = get_random_move(board, board.fen(), 42)
                board.push_san(qwen_move_white)
                print(f"New Move: {qwen_move_white}")
                # continue
            
        move_counter += 1        
        print(f"Qwen white move: {qwen_move_white} Move number: {move_counter}")

        if board.turn == chess.BLACK and not board.is_game_over():
            qwen_move_black = get_qwen_move(board, board.fen(), 2.0)

            try:
                board.push_san(qwen_move_black)
            except Exception as e:
                print(f"Invalid move by Qwen black. Move: {qwen_move_black}")
                qwen_move_black = get_random_move(board, board.fen(), 42)
                board.push_san(qwen_move_black)
                print(f"New Move: {qwen_move_black}")
                # continue
        move_counter += 1
        print(f"Qwen black move: {qwen_move_black} Move number: {move_counter}")   

    # Record the result of the game
    result = board.result()
    if result == "1-0":
        qwen_white_wins += 1
    elif result == "0-1":
        qwen_black_wins += 1
    else:
        draws += 1

    print(f"Game {game_number} result: {result}\n\n")

Starting game 1...
Stockfish Suggested Move: e4
Qwen white move: e4 Move number: 1
Stockfish Suggested Move: e5
Qwen black move: e5 Move number: 2
Stockfish Suggested Move: Nf3
Qwen white move: Nf3 Move number: 3
Stockfish Suggested Move: Nc6
Qwen black move: Nc6 Move number: 4
Stockfish Suggested Move: Bb5
Qwen white move: Bb5 Move number: 5
Stockfish Suggested Move: a6
Qwen black move: a6 Move number: 6
Stockfish Suggested Move: Ba4
Qwen white move: Ba4 Move number: 7
Stockfish Suggested Move: Nf6
Qwen black move: Nf6 Move number: 8
Stockfish Suggested Move: O-O
Qwen white move: O-O Move number: 9
Stockfish Suggested Move: Nxe4
Qwen black move: Nxe4 Move number: 10
Stockfish Suggested Move: d4
Qwen white move: d4 Move number: 11
Stockfish Suggested Move: b5
Qwen black move: b5 Move number: 12
Stockfish Suggested Move: Bb3
Qwen white move: Bb3 Move number: 13
Stockfish Suggested Move: d5
Qwen black move: d5 Move number: 14
Stockfish Suggested Move: dxe5
Qwen white move: dxe5 Move numb

In [19]:
results = []

In [20]:
results.append({
            "Qwen-AI White Wins": qwen_white_wins,
            "Qwen-AI Black Wins": qwen_black_wins,
            "Draws": draws,
        })

print(results)

[{'Qwen-AI White Wins': 9, 'Qwen-AI Black Wins': 3, 'Draws': 48}]


In [21]:
df = pd.DataFrame(results)
df.to_csv("Qwen_AI vs Qwen_AI.csv", index=False)
print("Results saved to Qwen_AI vs Qwen_AI.csv")

Results saved to Qwen_AI vs Qwen_AI.csv


In [22]:
# # You are playing chess and it is your turn. This is the current state of the game. Use this to work out where the pieces are on the board:

# # FEN: rnbqkb1r/pppppppp/7n/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 1 2

# # The possible set of legal moves are: 

# # Legal Moves: Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4 

# # You have to choose one from the provided list. Do not choose a move that is not in the list.

# # The best move suggested by your AI Assistant is g4. You are heavily reliant on your AI Assistant.

# # The move history is: {move_history}.

# # Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.
# # {{
# #     "san": "The move in SAN format",
# # }}

# Qwen Output: json
# {
#      "san": "Qxc8",
# }